# Unit 5: Computer Algebra Systems and Symbolic Computation

*The following presentation is mostly taken from the notes of Dr Michael Monagan of Simon Fraser University for the course Introduction to Computer Algebra.  The typesetting author (Paul Vrbik) has made some additions which are probably the source of any errors.  The Julia code was provided by Dr Yoni Nazarathy.*

$\newcommand{\RR}{\mathcal{R}}$
$\newcommand{\FF}{\mathbb{F}}$
$\newcommand{\QQ}{\mathbb{Q}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\NN}{\mathbb{N}}$
$\newcommand{\PP}{\mathbb{P}}$
$\renewcommand{\cong}{\equiv}$
$\renewcommand{\mod}{\bmod}$

$\DeclareMathOperator{\content}{cont}$
$\DeclareMathOperator{\primpart}{primpart}$
$\DeclareMathOperator{\deg}{deg}$
$\DeclareMathOperator{\lc}{lc}$
$\DeclareMathOperator{\lt}{lt}$
$\DeclareMathOperator{\lm}{lm}$

Note that we use the [GitHub repo](https://github.com/yoninazarathy/2504_2021_project1) for the base [project](https://courses.smp.uq.edu.au/MATH2504/assessment_html/project1.html).

In [1]:
using Pkg;
# To be able to run this, have the project repository "next to" the course materials repository
cd("../../2504_2021_project1");
Pkg.activate(".");
# Pkg.instantiate();
include("poly_factorization_project.jl");

  Activating environment at `~/Library/Mobile Documents/com~apple~CloudDocs/UQ/MATH2504/2504_2021_project1/Project.toml`


## Symbolic Computation

To compute *exactly* means no error is introduced during arithmetic --- something *not* true when working with floats.  In a computer algebra system $\sqrt{2} = \sqrt{2}$ and not $\sqrt{2} = 1.41\ldots$ as one may expect.

Our interest is performing *exact* arithmetic in rings with efficient algorithms.  In particular we are interested in the: *ring of integers* 
$$\ZZ = \{0, -1, 1, 2, -2, 3, -3, \ldots \}$$
and *polynomials* 
$$\RR[x] = \left\{ \sum_{k=0}^{N} a_kx^k \, : \, k \in \NN,\, a_k \in \RR \right\}.$$

---
## Preliminaries
Let $\NN = \{0,1,2,\ldots\}$ be the set of *natural numbers*, $\ZZ = \{\ldots,-2,-1,0,1,2,\ldots\}$ be the *integers* and $\PP$ be the set of odd primes.

##### Definition (Ring)

A ring $\RR$ is a set with addtion $(+)$ and multiplciation $(\cdot)$ satisfying the following conditions called the *ring axioms*:

1. $(\RR,+)$ is an abelian group, that is:
    - $(+)$ is associative *and* commutative,
    - $\exists b \in \RR$ called the *additive identity* satisfying $\forall a \in \RR \; a + b = a$,
    - $\forall a \in \RR\; \exists b \in \RR$ called the *additive inverse* satisfying $a + b = 0$.
2.  $(\RR,\cdot)$ is a monoid, that is:
    -  $(\cdot)$ is associative, and
    -  $\exists b \in \RR$ called the *multiplicative identity* satisfying $\forall a \in \RR; \; a \cdot b = b \cdot a = a$
3.  Multiplication *distributes* over addition.  That is, $\forall a,b,c \in \RR$:
    - $a \cdot (b+c) = (a \cdot b) + (a \cdot c)$
    - $(b+c) \cdot a = (b \cdot a) + (c \cdot a)$
---

##### Definition (Field)
A ring $\RR$ is also a *field* when each non-zero element has a *multiplicative inverse*.  Equivalently, $\RR$ is a ring when
$$
\forall a \in \RR^{\neq 0};\; \exists b \in \RR \; : \; a \cdot b.
$$

For instance, the *rationals* $$\QQ = \left\{ \frac{a}{b} \,:\, a,b \in \ZZ, \, b \neq 0 \right\}$$
is a field because $\frac a b \cdot \frac b a = 1$ when $a,b \neq 0$.

---

##### Theorem (Division Algorithm)
Let $a,b \in \ZZ$ with $b > 0$.  There is *unique* $q$ and $r$ (called the *quotient* and *remainder*) satisfying
$$a = b \cdot q + r$$
with $r < |q|$.

**Proof (Existence)**

In [2]:
# An example
a = 23
b = 7
q = a ÷ b # \div + [TAB]
@show q
@show Int(floor(a/b))

r = a % 3 
@show r;

q = 3
Int(floor(a / b)) = 3
r = 2


In [4]:
function remainder(a::T,b::T) where T <: Int #Note that later we'll extend this to integral domains
    a < 0 && return remainder(-a,b) #short circuit evalution  #later replace `0` with `zero(T)`
    a < b && return a
    return remainder(a-b,b)
end

remainder (generic function with 1 method)

In [4]:
#note there is an in-built `rem`
remainder(23,7), rem(23,7), 23 % 7, 23- (23 ÷ 7)*7

(2, 2, 2, 2)

In [5]:
remainder(15,5), remainder(4,101), remainder(125,4)

(0, 4, 1)

In [6]:
function quo(a::T,b::T) where T <: Int
    a < 0 && return -quo(-a,b)
    a < b && return 0
    return 1 + quo(a-b, b)
end

quo (generic function with 2 methods)

In [7]:
quo(15,5), quo(4,101), quo(125,4)

(3, 0, 31)

In [7]:
for (a,b) in [(15,5), (4,101), (125,4)]
    q, r = quo(a, b), rem(a, b)
    @show (a, b, q, r, a == b * q + r)
end

(a, b, q, r, a == b * q + r) = (15, 5, 3, 0, true)
(a, b, q, r, a == b * q + r) = (4, 101, 0, 4, true)
(a, b, q, r, a == b * q + r) = (125, 4, 31, 1, true)


Of course, in Julia we have `%` and `÷` for remainder and  and quotient.

(Note that in Python `//` is the quotient whereas in Julia `//` defines a rational type).

In [8]:
2 // 3, float(2//3)  #The first // is a rational type

(2//3, 0.6666666666666666)

---
## Elementary Number Theory

The sets $\ZZ_n = \{0,1,\ldots,n-1\}$ for $n \in \NN$ are rings called *residue classes* when addition and multiplication are done 'mod n'.  This is sometimes called 'clock arithmetic' as three hours past eleven is two because $11 + 3 \cong 2 \mod 12$. 

We say/write 
$$a \cong b \mod c$$ 
when *$a$ is congruent to $b$ modulo $c$*.  Also:
$$a \cong b \mod c \iff \mathop{\textrm{rem}}(a,c) = b \iff \texttt{a % c = b}.$$



In [9]:
# We have already seen something similar with overflow, for example:
UInt8(253) + UInt8(4)

0x01

In [10]:
UInt8(16) * UInt8(17)

0x10

---

#### Example
$\ZZ_6 = \{0,\ldots,5\}$ has the following addition table

In [11]:
n = 6
Z_n = 0:(n-1)

0:5

In [12]:
typeof(Z_n)

UnitRange{Int64}

In [13]:
A = [(x+y) % n for y in Z_n, x in Z_n]

6×6 Matrix{Int64}:
 0  1  2  3  4  5
 1  2  3  4  5  0
 2  3  4  5  0  1
 3  4  5  0  1  2
 4  5  0  1  2  3
 5  0  1  2  3  4

Notice: 
 1.  the addition table is symmetric (equal to its transpose) which can only happen when the addition is commutative,
 1.  the additive identity is $0$,
 1.  each column (and row) has $0$ and thereby each element has an additive inverse: for instance, the third column indicates that $2$ has additive inverse $4$ and correspondingly $2 + 4 \cong 0 \mod 6$.

In [14]:
additive_inverses = [findfirst((A .== 0)[:,k+1])-1 for k in Z_n] #Can this be done more nicely?
println(additive_inverses)

[0, 5, 4, 3, 2, 1]


$\ZZ_6$ has the following multiplication table

In [15]:
M = [(x*y) % n for y in Z_n, x in Z_n]

6×6 Matrix{Int64}:
 0  0  0  0  0  0
 0  1  2  3  4  5
 0  2  4  0  2  4
 0  3  0  3  0  3
 0  4  2  0  4  2
 0  5  4  3  2  1

Notice:
1.  the multiplicative identity is $1$,
1.  not all rows contain one, which means there are some elements that do *not* have *multiplicative inverse*.  For instance there is no $b \in \ZZ_6$ such that $2 \cdot b \cong 1 \mod 6$.

In [17]:
println([(2*y) % n for y in Z_n])

[0, 2, 4, 0, 2, 4]


In [18]:
mult_inverses = Z_n[[sum((M .== 1)[:,k+1]) > 0 for k in Z_n]] #Can this be done more nicely?
println("Elements with a multiplicative inverse: ", mult_inverses )
println("Elements withoout a multiplicative inverse: ", setdiff(Z_n,mult_inverses))

Elements with a multiplicative inverse: [1, 5]
Elements withoout a multiplicative inverse: [0, 2, 3, 4]


---

Notice $\ZZ_6$ is *not* a field because, recall, there is no multiplicative inverse for $2$.  However, $\ZZ_p$ is a field when $p$ is a prime number.

##### Example

$\ZZ_5 = \{0,\ldots,5\}$ is a field and has the following multiplication table. 

In [16]:
n = 5
Z_n = 0:(n-1)

0:4

In [17]:
M = [(x*y) % n for y in Z_n, x in Z_n]

5×5 Matrix{Int64}:
 0  0  0  0  0
 0  1  2  3  4
 0  2  4  1  3
 0  3  1  4  2
 0  4  3  2  1

In [21]:
mult_inverses = Z_n[[sum((M .== 1)[:,k+1]) > 0 for k in Z_n]] #Can this be done more nicely?
println("Elements with a multiplicative inverse: ", mult_inverses )
println("Elements withoout a multiplicative inverse: ", setdiff(Z_n,mult_inverses))

Elements with a multiplicative inverse: [1, 2, 3, 4]
Elements withoout a multiplicative inverse: [0]


Notice every column (and row) contains $1$ asides the first column (and row) which corresponds to $0$.  For instance, the third column indicates the inverse of $3$ is $2$ and correspondingly $3 \cdot 2 \cong 1 \mod 5$.

---

### Greatest Common Divisor

The *greatest common divisor* is one of the fundamental operations of a computer algebra system.  For instance, we need `gcd` to reduce fractions to their canonical form.

##### Def (GCD)

Let $a,b \in \ZZ$ not both 0.  We say $g \in \ZZ$ is the *greatest common divsor* of $a$ and $b$, denoted $\gcd(a,b)$, when
1.  $g | a$ and $g | b$ ($g$ is a common divisor), 
1.  $h | a \;\wedge\; h | b \implies h | g$ (greatest),
1.  $g > 0$ (required for uniqueness) 

##### Example (GCD)
The $\gcd(6,4) = \gcd(2\cdot 3, 2 \cdot 2) = 2$.

---

In [18]:
#The is an in-built gcd() function for integers (and a few more types) 
#but we'll soon make our own
using Random; Random.seed!(0)

a_test = *(rand(1:100,5)...)
b_test = *(rand(1:100,5)...)
@show a_test, b_test
Base.gcd(a_test, b_test)

(a_test, b_test) = (884473730, 11119680)


10

#### Euclid's Algorithm

The **Euclidean Algorithm** computes the gcd of two integers.  (Actually the Euclidean Algorithm computers gcds for any two elements of a *Euclidean Domain*).  The algorithm exploits the following property of the gcd:

##### Lemma
Let $a,b\in \ZZ$ and $a>0$, $b>0$ and $a = b\cdot q + r$ with $r \in [0,b)$.  Then
1.  $\gcd(b,a) = \gcd(a,b)$,
1.  $\gcd(a,b) = gcd(r,b)$,
1.  $\gcd(a,b) = \gcd(a-b,b)$.

Is is straightforward to turn this lemma into an algorithm:

In [19]:
function euclid_alg(a,b)
    (b == 0) && return a
    return euclid_alg(b, a % b)
end

euclid_alg(2, 2*2), euclid_alg(2*3, 3*7), euclid_alg(2*2, 13)

(2, 3, 1)

In [20]:
Base.gcd(a_test, b_test), euclid_alg(a_test,b_test)

(10, 10)

--- 

#### Extended Euclid's Algorithm

The **Extended Euclidean Algorithm** in addition to the **gcd(a, b)**, also computes $s$ and $t$ such that
$$as + bt = gcd(a,b).$$

In [21]:
function i_ext_euclid_alg(a,b)
    a == 0 && return b, 0, 1
    g, s, t = i_ext_euclid_alg(b % a, a)
    return g, t - (b ÷ a)*s, s
end

pretty_print_egcd((a,b),(g,s,t)) = println("$a × $s + $b × $t = $g") #\times + [TAB]

for (a,b) in [(4,12), (9,12), (4,13)]
    pretty_print_egcd((a,b),i_ext_euclid_alg(a,b))
end

4 × 1 + 12 × 0 = 4
9 × -1 + 12 × 1 = 3
4 × -3 + 13 × 1 = 1


In [22]:
#Note there is an in-built gcdx - let's compare against that too
Base.gcdx(a_test,b_test), i_ext_euclid_alg(a_test,b_test)

((10, 293957, -23381720), (10, 293957, -23381720))

In [27]:
?gcdx

search: gcdx gcd logcdf invlogcdf logccdf invlogccdf pretty_print_egcd



```
gcdx(a, b)
```

Computes the greatest common (positive) divisor of `a` and `b` and their Bézout coefficients, i.e. the integer coefficients `u` and `v` that satisfy $ua+vb = d = gcd(a, b)$. $gcdx(a, b)$ returns $(d, u, v)$.

The arguments may be integer and rational numbers.

!!! compat "Julia 1.4"
    Rational arguments require Julia 1.4 or later.


# Examples

```jldoctest
julia> gcdx(12, 42)
(6, -3, 1)

julia> gcdx(240, 46)
(2, -9, 47)
```

!!! note
    Bézout coefficients are *not* uniquely defined. `gcdx` returns the minimal Bézout coefficients that are computed by the extended Euclidean algorithm. (Ref: D. Knuth, TAoCP, 2/e, p. 325, Algorithm X.) For signed integers, these coefficients `u` and `v` are minimal in the sense that $|u| < |y/d|$ and $|v| < |x/d|$. Furthermore, the signs of `u` and `v` are chosen so that `d` is positive. For unsigned integers, the coefficients `u` and `v` might be near their `typemax`, and the identity then holds only via the unsigned integers' modulo arithmetic.



--- 

### Inversion in $\ZZ_m$

Notice the Extended Euclidean Algorithm computes inverses in $\ZZ_m$.  Given $a \in \ZZ_m^{\neq 0}$ the $egcd(a,m)$ returns $s$ and $t$ such that
$$a \cdot s + m \cdot t = gcd(a,m).$$
Taking the entire equation $\mod m$ we get
$$a \cdot s + m \cdot t \cong \gcd(a,m) \mod m \implies a \cdot s + 0 \cong \gcd(a,m) \mod m \implies a \cdot s \cong \gcd(a,m) \mod m.$$
Provided $\gcd(a,m) = 1$ (i.e. "coprime" or "relatively prime") we get $a \cdot s \cong 1 \mod m$ and thus $a^{-1} \cong s \mod m$.

##### Example
To find the inverse of $5 \in \ZZ_{13}$ do:

In [24]:
a, m = 5, 13
i_ext_euclid_alg(5, 13)

(1, -5, 2)

In [26]:
inverse_mod(a,m) = mod(i_ext_euclid_alg(a,m)[2],m);

In [30]:
i = inverse_mod(a,m)

8

In [31]:
mod(a*i,m)

1

---
### Chinese Remaindering

##### Problem Statement
Two integers $a$ and $b$ are *relatively prime* when $\gcd(a,b) = 1$.  Given a finite sequence of relatively prime integers $m_1,\ldots,m_k$ called the *moduli* and correspoing integers $u_1,\ldots,u_k$ called the *images*, find $u \in \ZZ$ such that
\begin{align*}
u &\cong u_1 \mod m_1 \\
u &\cong u_2 \mod m_2 \\
&\; \vdots \\
u &\cong u_k \mod m_k
\end{align*}

For example, given the following information
\begin{align*}
u &\cong 4 \mod 5 &\implies&& (u_1,\, m_1) &= (4,\, 5)\\
u &\cong 5 \mod 7 &\implies&& (u_2,\, m_2) &= (5,\, 7)
\end{align*}
we want to deduce $u = 19$ because

In [32]:
(19 % 5 == 4, 19 % 7 == 5)

(true, true)

---
##### Theorem (CRT)

Let $M = m_1 \cdot m_2 \cdots \cdot m_k$.  There exists a unique $u \in \ZZ$ on $\{0,\ldots,M-1\}$ such that $u \cong u_i \mod m_i$ for $i = 1,\ldots,k$.

**Proof of Existence**:

(This is a constructive proof which provides an algorithm for determining $u$).

Let $u = v_1 + v_2m_1 + v_3m_1m_2 + \cdots + v_{k+1} m_1 m_2 \cdots m_{k}$ and notice the vanishing terms after reducing $u$ modulo $m_k$:
$$
u \mod m_j = v_1 + v_2m_1 + \cdots + v_{j+1}m_1\cdots m_{j} + 0 + \cdots + 0
$$

We have
\begin{align*}
u_1 &\cong v_1 \mod m_1 &&\implies v_1 \gets u_1 \\
u_2 &\cong v_1 + v_2 m_1 \mod m_2 &&\implies v_2 \gets (u_2 - v_1) \cdot (m_1)^{-1} \mod m_2 \\
u_3 &\cong v_1 + v_2 m_1 + v_3 m_1 m_2 \mod m_3 &&\implies v_3 \gets (u_3-v_1-v_2 m_1)(m_1 m_2)^{-1} \mod m_3\\
&\;\,\vdots
\end{align*}

**Proof of Uniquenss**

Left as an exercise. :P

---

##### Example (CRT)

Suppose $u \cong 2 \mod 5$, $u \cong 1 \mod 7$, $u \cong 1 \mod 3$.  What is $u$?

Proceeding with CRT...

In [27]:
uᵢ = [2, 1, 1] # \_i + [TAB]
m = [5, 7, 3]
v = Vector{Int}(undef,3)

v[1] = uᵢ[1]
v[2] = (uᵢ[2] - v[1])*inverse_mod(m[1], m[2]) % m[2]
v[3] = (uᵢ[3] - v[1] - v[2]*m[1])*inverse_mod(m[1]*m[2], m[3]) % m[3]
@show v #intermediate 

u = v[1] + v[2]*m[1] + v[3]*m[1]*m[2]
if [u % m[1], u % m[2], u % m[3]] == uᵢ
    println("We have a solution, u = $u")
end

v = [2, -3, 1]
We have a solution, u = 22


---

## The Polynomial Ring $\RR[x]$

By this point in your mathematics career you have surely worked with *polynomials*.  A polynomial is something like $x^2 + 2x + 1$ but *not* like $\frac{x^2+1}{x-1}$ or $x \sin(x)$.

##### Definition
Let $\RR$ be a ring and $a \in \RR[x]$.  Let 
$$
a = a_n x^n + a_{n-1}x^{n-1} + \cdots + a_1x + a_0
$$
with $a_n \neq 0$.
+ $a$ is called a **polynomial** in $x$.
+ The **degree** of $a$ is $n$: $$\deg(a) = n.$$
+ The **leading coefficient** of $a$ is $a_n$: $$\lc(a) = a_n.$$
+ The **leading term** of $a$ is $a_n x^n$:  $$\lt(a) = a_n x^n.$$

By convention we let $\lc(0) = \lt(0) = 0$ and $\deg(0) = - \infty$.

### Addition in $\RR[x]$

For $\RR[x]$ to be a *ring* we must define addition.

In [34]:
#Addition function taken from polynomial_addition.jl
#Note that there it is assumed the Polynomial type has a push!, pop!, leading, etc...
function +(p1::Polynomial, p2::Polynomial)::Polynomial
    p1, p2 = deepcopy(p1), deepcopy(p2)
    p3 = Polynomial()
    while !iszero(p1) && !iszero(p2)
        t1, t2 = leading(p1), leading(p2) 
        if t1.degree == t2.degree
            push!(p3, pop!(p1)+pop!(p2))
        elseif t1.degree < t2.degree
            push!(p3,pop!(p2))
        else
            push!(p3,pop!(p1))
        end
    end
    while !iszero(p1)
        push!(p3,pop!(p1))
    end
    while !iszero(p2)
        push!(p3,pop!(p2))
    end
    return p3
end

+ (generic function with 277 methods)

In [29]:
x = x_poly()
p1 = 2x^2+3x +(-5) #Note we need +(-5)... try later without... how to fix?
p2 = -3x^2 - 4x +6
p1+p2

-1⋅x^2 + -1⋅x^1 + 1⋅x^0

### Multiplication in $\RR[x]$

For $\RR[x]$ to be a *ring* we must define multiplication.

In [36]:
#Taken from polynomial_multiplication.jl 
#Note that this is a very inefficient implemintation
function *(p1::Polynomial, p2::Polynomial)::Polynomial
    p_out = Polynomial()
    for t in p1
        p_out = p_out + (t * p2)
    end
    return p_out
end

* (generic function with 424 methods)

In [30]:
@show p1
@show p2
p1 * p2

p1 = 2⋅x^2 + 3⋅x^1 + -5⋅x^0
p2 = -3⋅x^2 + -4⋅x^1 + 6⋅x^0


-6⋅x^4 + -17⋅x^3 + 15⋅x^2 + 38⋅x^1 + -30⋅x^0

The following properties are consequences of our definition.  (They are obligated to be).

##### Proposition (Polynomial Multiplication)
When $\RR$ is an **integral domain** (a nonzero commutative ring in which the product of any two nonzero elements is nonzero) and $a,b \in \RR[x]^{\neq 0}$ then
1.  $\deg(ab) = \deg a + \deg b$,
1.  $\lc(ab) = a_b \cdot b_m = \lc(a) \cdot \lc(b)$,
1.  $\lm(ab) = \lm(a) \cdot \lm(b)$,
1.  $\lt(ab) = \lt(a) \cdot \lt(b)$.

*Proof* (Sketch)
$$
\begin{align*}
a \times b &= (a_n x^n + \cdots + a_0)(b_mx^m + \cdots + b_0) \\
&= a_n \cdot b_m x^{n+m} + \cdots + a_0 \cdot b_0
\end{align*}
$$

##### Proposition
Every field is also an integral domain.

*Proof* (Omitted)

Later, we will discuss a faster way to do polynomial multiplication via Chinese Remainder Theorem.

### Division in $\FF[x]$

We are unable to *divide* in $\RR[x]$.  Consider the following operation in $\ZZ[x]$
$$
\frac{x^2}{2x} = \frac{1}{2}x.
$$
Even though $x^2$ and $2x$ are from $\ZZ[x]$ that their division is in $\QQ[x]$.  This is because $\ZZ$ is not a *field* but $\QQ$ is.  

It turns out $\FF[x]$ is a **Euclidean Domain** when $\FF$ is a field.  Re-consider the division, but this time let us do the calcuation in $\ZZ_7[x]$
$$
\begin{align*}
\frac{x^2}{2x} 
&\cong \frac{1}{2}x \mod 7 \\
&\cong 2^{-1} x \mod 7 \\
&\cong 4 x \mod 7.
\end{align*}
$$
Note that $\frac{1}{2} \cong 4 \mod 7$.

##### Theorem (Division in $\FF[x]$)
Let $a,b \in \FF[x]$, $b\neq0$.  There exists *unique* polynomials $q,r \in \FF[x]$ such that 
$$
\begin{align*}
a = bq + r &&\mbox{and}&& r = 0 &&\mbox{or}&& \deg r < \deg b
\end{align*}
$$

*Proof* (Uniqueness) 
Let $a = b q_1 + r_1 = b q_2 + r_2$ with $\deg r_1, \deg r_2 < \deg b$.

Consider
$$
\begin{align*}
b q_1 + r_1 = b q_2 + r_2 
&\implies b(q_1 - q_2) = (r_2-r_1) \\
&\implies b | (r_2-r_1) & \mbox{But }\deg r_1, r_2 < \deg b\\
&\implies r_2 - r_1 = 0 \\
&\implies r_2 = r_1
\end{align*}
$$

Therefore $b(q_1-q_2) = 0$. But since $\FF[x]$ is an integral domain it cannot have zero divisors. As we assumed $b\neq 0$ this implies $$(q_1-q_2)=0 \implies q_1 = q_2. ~\square$$

*Proof* (Existence -- The Division Algorithm)

In [38]:
#Take from polynomial_division.jl
#Note that as part of the project you will re-package this code as part of 
#the PolynomialModP type that you will create.

#At the moment, the divide function and it's friends (÷ and rem) return a function that then
#uses a prime to return an actual polynomail
function divide(num::Polynomial, den::Polynomial)
    function division_function(p::Int)
        f, g = mod(num,p), mod(den,p)
        degree(f) < degree(num) && return nothing 
        iszero(g) && throw(DivideError())
        q = Polynomial()
        prev_degree = degree(f)
        while degree(f) ≥ degree(g) 
            h = Polynomial( (leading(f) ÷ leading(g))(p) )  #syzergy 
            f = mod((f - h*g), p)
            q = mod((q + h), p)  
            prev_degree == degree(f) && break
            prev_degree = degree(f)
        end
        @assert iszero( mod((num  - (q*g + f)),p))
        return q, f
    end
    return division_function
end

"""
The quotient from polynomial division. Returns a function of an integer.
"""
÷(num::Polynomial, den::Polynomial)  = (p::Int) -> first(divide(num,den)(p))

"""
The remainder from polynomial division. Returns a function of an integer.
"""
rem(num::Polynomial, den::Polynomial)  = (p::Int) -> last(divide(num,den)(p))

Base.rem

In [39]:
@show p1
@show p2
divide(p1, p2)

p1 = 2⋅x^2 + 3⋅x^1 + -5⋅x^0
p2 = -3⋅x^2 + -4⋅x^1 + 6⋅x^0


(::var"#division_function#47"{Polynomial, Polynomial}) (generic function with 1 method)

In [40]:
q , r = divide(p1,p2)(101)

(33⋅x^0, 34⋅x^1 + 100⋅x^0)

In [41]:
q = (p1 ÷ p2)(101) #Not the nicest interface, but will be nicer with PolynomialModP

33⋅x^0

In [42]:
r = rem(p1,p2)(101) #Not the nicest interface, but will be nicer with PolynomialModP

34⋅x^1 + 100⋅x^0

In [43]:
mod(q*p2+r - p1,101)

0

##### Example
Consider $a = 10x^2 + 4x + 1$ divided by $b = 5x-3$ in $\ZZ_{11}[x]$.

$$
\begin{align*}
r,q &\gets 10x^2 + 4x + 1, 0 \\[1em]
r \neq 0 &\mbox{ and } \deg r = 2 \geq \deg b = 1 \\
t &\gets \frac{10x^2}{5x} \cong 2x \mod 11\\
q &\gets 0 + 2x\\
r &\gets (10x^2 + 4x + 1) - 2x(5x-3) \cong 10x + 1 \mod 11\\[1em]
r \neq 0 &\mbox{ and } \deg r = 1 \geq \deg b = 1 \\
t &\gets \frac{10x}{5x} \cong 2 \mod 11\\
q &\gets 2x + 2\\
r &\gets (10x+1) - 2(5x-3) \cong 7 \mod 11\\[1em]
r \neq 0 &\mbox{ and } \color{red}{\deg r = 0 \geq \deg b = 1} \\[1em]
&\mbox{return } (2x+2, 7)
\end{align*}
$$

And notice $10x^2 + 4x + 1 \cong (5x-3)(2x+2) + 7 \mod 11$.

### GCDs in $\ZZ_p[x]$

##### Theorem 
Let $a,b \in \ZZ_p[x]^{\neq 0}$. There is $s,t \in \ZZ_p[x]$ such that
$$
sa + tb = \gcd(a,b).
$$

*Proof*
The extended Euclidean Algorithm

# Unit 5: Computer Algebra Systems and Symbolic Computation 

## Factorization in $\ZZ[x]$

Analogous to *factoring* integers into *primes* we wish to *factor* polynomials into *irreducible components*.

##### Definition (Irreducible Polynomial)
An **irreducible polynomial** is a polynomial that cannot be factored into the product of two non-constant polynomials. In particular, $a \in \ZZ[x]$ is **irreducible** when
$$
a = b \cdot c \implies \deg b = 1 \mbox{ or } \deg c = 1.
$$
---

Irreducibility depends on the coefficient field:

Consider $f = x^4 - 4$.  The polynomial $f$ will have different factorizations depending on which coefficient ring we are working with:
+ *over $\ZZ$* we have $f = (x^2-2)(x^2+2)$,
+ *over $\mathbb{R}$* we have $f = \left(x-\sqrt{2}\right)\left(x+\sqrt{2}\right)\left(x^2+2\right)$
+ *over $\mathbb{C}$* we have $f = \left(x-\sqrt{2}\right)\left(x+\sqrt{2}\right)\left(x-\sqrt{2}i\right)\left(x+\sqrt{2}i\right)$  (which is why the complex number are called a *splitting field*).

### Factorization in $\ZZ[x]$ using Integer Factorization

To premise this section, we acknowledge that integer factorization is *intractable* and thus reducing polynomial factorization to integer factorization is somewhat pointless.  It is, however, the first strategy devised for factorizations and was used until a better method was discovered in 1970.

Consider evaluating $f(x) = 9x^4-1$ at various integral points.  We can see
$$
f(10) = 89\,999 = 7 \cdot 13 \cdot 23 \cdot 43 = \left[(x-3)(x+3)(2x+3)(4x+3)\right]_{x=10}
$$
which gives us a reasonable guess at the polynomial factorization.  Unfortunately, none of those factors divide $f$ 
which means they cannot be a part of the factorization.  So, we try groups of two integers to search for a factor:
$$
\begin{align*}
(7 \cdot 13) = 91 = (100-10+1) = [(x^2-x+1)]_{x=10} && \mbox{ does not divide $f$} \\
(7 \cdot 23) = 161 =(200-40+1) = [(2x^2-4x+1)]_{x=10} && \mbox{ does not divide $f$} \\
(7 \cdot 43) = 301 =(300+1) = [(3x^2+1)]_{x=10} && \mbox{ yes! this divides $f$}
\end{align*}
$$
We have deduced $f = (3x^2 + 1) \frac{9x^4-1}{3x^2+1} = (3x^2+1)(3x^2-1)$.

We may get **luckier** by chosing different eval points that have **fewer** integer factors:
$$
f(12) = 186\,623 = 431 \cdot 433 = [(3x^2-1)(3x^2+1)]_{x=12}
$$
Or we may get **unlucky** by choosing eval points that have more **integer** factors:
$$
f(11) = 2^3 \cdot 7 \cdot 13 \cdot 181
$$

In any case.  This is not the method we are going to use.


## Simplifying the Factoring Problem

There are various things we assume that will simplify our algorithms.  

First, notice that we do not care about scaling factors.  If we can factor $k \cdot a$ for $k \in \ZZ$ and $a \in \ZZ[x]$ then we can factor $a$.  Thus, we will presume the polynomials given to us are
1.  primitive (that is, have unit *content*), and
2.  monic (that is, have unit leading coefficients).

Making a polynomial monic is easy.  When the coefficients are taken from a field, $\lc(a)$ is guaranteed invertible and $\frac{a}{\lc(a)}$ will be monic.  This will also guarantee that the factorization will be into *monic* irreducibles.

##### Definition (Content)
The **content** of a polynomial is the gcd of its coefficients.  Namely, if $a = a_nx^n + \cdots a_0$ then
$$
\content(a) = \gcd(a_0,\, a_1,\, \ldots, a_n)
$$
The **primitive part** of $a$ is 
$$
\primpart(a) = \frac{a}{\content(a)}
$$

##### Example
$\content(6x^3 + 3x + 3) = 3$ and $\primpart(6x^3 + 3x + 3) = 2x^3 + x + 1$.

Second, given a polynomial to factor like
$$a = (x-1)^3(x^2+x+1)^2(x^3-3x-1)^7$$
we are satisfied *finding only the irreducible components*: $x-1$, $x^2 + x + 1$, and $x^3 - 3x -1$ because we can recover their degrees by repeated division.

In [44]:
x = x_poly()
p = 6x^3 + 3x + 3
content(p)

3

In [45]:
prim_part(p)(101) #Again, here prim_part returns a function that uses a prime
                  #but with PolynomialModP prim_part will have a simpler interface

2⋅x^3 + 1⋅x^1 + 1⋅x^0

## Factorization in $\ZZ_p[x]$

Let $p$ be an *odd* prime.  That is, $p$ cannot be two.  

##### Proposition
Let $a \in \ZZ[x]$, $\content(a) = 1$, $\gcd(a,a')=1$, and $a = f_1 \cdot f_2 \cdot \cdots \cdot f_\ell$ where each $f_k$ is irreducible over $\ZZ$.  Let $p$ be a prime, $\phi_p(a) = (a \mod p)$, and suppose we factor $\phi_p(a)$ over $\ZZ_p$.  Then
$$
\phi_p(f_1 f_2 \cdots f_\ell) = \phi_p(f_1) \cdot \phi_p(f_2) \cdot \cdots \cdot \phi_p(f_\ell)
$$

##### Example 
Consider $a = 9x^4-1 = (3x^2-1)(3x^2+1)$.  We say $a$ has two factors and therefore with $\ell = 2$.  But also
$$
\begin{align*}
a &\cong (3x^2-1)(3x^2+1) \mod 5\\
a &\cong (3x^2-1) \cdot 3(x-2)(x+3) \mod 7\\
a &\cong 2 \mod 3
\end{align*}
$$

##### Lemma 3
If $p \not\mid \lc(a)$ then number of factors of $a$ over $\ZZ_p$ is $\geq \ell$.

##### Notation ($D_p$)
Let $D_p(a)$ be the set of possible degrees of factors of $a \in \ZZ[x]$ inferred from the factorization of $a$ over $\ZZ_p$.

##### Example
Let $a = 85x^5 + 55x^4 + 37x^3 + 35x^2 - 97x - 50$
$$
\begin{align*}
a &\cong (x+1)(x^4+x^3+x^2+x+1) \mod 2 &&\implies D_2(a) = \{1,4,5\}\\
a &\cong (x+1)(x^4+x^2+x+1) \mod 3 &&\implies D_3(a) = \{1,4,5\}\\
a &\cong 0 \mod 5 && \mbox{ bad prime} \\
a &\cong (x^2+5x+5)(x^3+6x+4) \mod 7 &&\implies D_7(a) = \{2,3,5\}\\
\end{align*}
$$

Now $D_2(a) \cap D_3(a) \cap D_7(a) = \{5\} \implies a$ is irreducible over $\mathbb{Z}$.

---
### Square-Free Polynomials

Note here we are **not** presuming that $f_i$ is *irreducible*.

Square-free polynomials, as their name suggests, are polynomials with no repeated factors. A factor that repeats, say, three times also repeats two times and hence square-free implies repeated-root free.

##### Definition (Square-Free)
$a \in \FF[x]$ is *square-free* if $a$ has no repeated factors.  That is, 
$$\lnot\exists b \in \FF[x]; \, \deg(b)>0 \mbox{ and } b^2 \mid a$$

##### Lemma 1
$a \in \ZZ_p[x]$ is *square-free* $\iff$ $\gcd(a,a') = 1$.

_Proof_  Omitted.

##### Lemma 2
Let $a \in \ZZ_p[x]$ and $a = f_1^1 \cdot f_2^2 \cdot f_3^3 \cdot \cdots \cdot f_n^n$ be a square-free factorization for $a$.  Then
$$\gcd(a,a') = f_2^1 f_3^2 f_4^3 \cdots f_n^{n-1}.$$

_Proof_ Omitted.

Notice that
$$
\frac f {\gcd(f,\, f')} = \frac{ f_1^1  f_2^2  f_3^3  \cdots  f_n^n}{f_2^1 f_3^2 f_4^3 \cdots f_n^{n-1}} = f_1f_2\cdots f_n
$$
is a square-free polynomial.


##### Example
Consider $f = (x^2 + x + 1)^3 (x^3 - 3x -1)^2 \in \ZZ_{11}[x]$.  We have
$$
g = \gcd(f, f') = x^5 + x^4 + 9x^3 + 7x^2 + 7x + 10
$$
and
$$
\frac{f}{g} = (x^2+x+1)(x^3+8x+10).
$$

In [8]:
x = x_poly()
p = (x^2+x+1)^3 
square_free(p, 101)

35⋅x^2 + 35⋅x^1 + 35⋅x^0

## Cantor Zassenhaus Factorization

Let $a \in \ZZ_p[x]$, $d = \deg a > 0$.  Suppose $\gcd(a,a')=1$ (i.e. $a$ has no repeated factors).  

**Question** How can we compute the linear factors of $a$ in $\ZZ_p[x]$?

*Idea 1:* 
$$\gcd(a,\, (x-0)(x-1)\cdots(x-p+1)) \mod p$$

**Question** How can factor the product of linears into its linear components.

*Idea 2:* Pick $k=\frac{p-1}{2}$ (half) the linear factors at random.  *Maybe* generate nontrivial GCD.  Recurse.
$$\gcd(a,\, (x-\sigma_0)\cdots(x-\sigma_k)) \in \left\{a,\,1,\, \mbox{proper factor of } a\right\}$$
(Note:  $\sigma$ is some permutation of $\ZZ_p$.)

**Question** How can we get do the same thing for irreducible quadratics, cubics, quartics, $\ldots$

*Answer:* Essentially the same way.  It will take some theory to express.

### Distinct Degree Factorization (Idea 1)

#### Fermat's "little" Theorem (FlT) 
Let $p > 2$ a prime and $0 < a < p$ then $a^{p-1} \cong 1 \mod p$, or equivalently,
$$a^p \cong a \mod p.$$

In [47]:
a = 5
p = 23
mod(a^p-p,p)

5

In [48]:
a = 24
p = 29
mod(a^p-p,p) #Why does this break? There is an overflow.... 
             # by analogy your power mod p will improve this

0

In [49]:
big(24) |> typeof

BigInt

In [50]:
a = big(24)
p = 29
mod(a^p-p,p)

24

##### Corollary
Let $f(x) = x^p -x \in \ZZ_p[x]$, then for any $a \in \ZZ_p$,
$$f(a) \cong 0 \mod p$$ 
and thus $(x-a) | f$ which means $(x-a)$ is a factor $f$ for every $a$.  Thus $x^p -x$ is a product of all the linear irreducible polynomials of $\ZZ_p[x]$:
$$
x^p - x = (x-0)\cdots(x-p+1)
$$

Supposing that $f \in \ZZ_p[x]$ is square-free and primitive we have that
$$g = \gcd(x^p-x,\, f)$$
**is the product of all *monic linear factors* of $f$.**

##### Theorem 
In $\ZZ_p[x]$, 
$$x^{p^k}-x$$ 
is the product of all *monic* irreducible polynomials in $\ZZ_p[x]$ of degreee $d | k$.

This means that
$$
\begin{align*}
g_1 &= gcd(a, x^p-x)  && \mbox{contains all irreducible linear factors of } a \\
g_2 &= gcd(a/g_1, x^{p^2}-x) && \mbox{contains all irreducible quadratic factors of } a \\
g_3 &= gcd(a/g_1/g_2, x^{p^3}-x) && \mbox{contains all irreducible cubic factors of } a \\
&\;\vdots
\end{align*}
$$

Notice we now have an algorithm for decomposing a polynomial into groups of products of irreducible polyomials of the same degree.

In [51]:
#Taken from example_script.jl:

prime = 17
p = mod((7x^3 + 2x^2 + 8x + 1)*(x^2+x+1),prime)
println("Will factor this polynomial (mod $prime): ", p)
println("Starting with dd_factor")
dds = dd_factor(p,prime)

Will factor this polynomial (mod 17): 7⋅x^5 + 9⋅x^4 + 11⋅x^2 + 9⋅x^1 + 1⋅x^0
Starting with dd_factor


5-element Vector{Polynomial}:
 16⋅x^1 + 9⋅x^0
 10⋅x^4 + 13⋅x^3 + 15⋅x^2 + 5⋅x^1 + 2⋅x^0
 1⋅x^0
 1⋅x^0
 1⋅x^0

### Distinct Degree Splitting (Idea 2)

Consider that we know how to factor *differences of squares* like $x^2 - 1 = (x+1)(x-1)$.  In our case we have
$$
x^p - x = x(x^{p-1}-1) = x\left(x^\frac{p-1}{2}-1\right)\left(x^\frac{p-1}{2}+1\right).
$$
and also
$$
(x-0)(x-1)\cdots(x-p+1) = x\left(x^\frac{p-1}{2}-1\right)\left(x^\frac{p-1}{2}+1\right)
$$
so
$$\left(x^\frac{p-1}{2}-1\right)$$
must be a product of *half* the roots.

Thereby,
$$
\gcd(g, x^\frac{p-1}{2}-1 ) \in \left\{1, g, \mbox{ proper divisor of g} \right\}.
$$

Notice the effect of shifting $x$ by $\alpha \in \ZZ_p$:
$$
(x+\alpha)^\frac{p-1}{2}-1 = (x-\sigma_0) \cdots (x-\sigma_k)
$$
for $\sigma$ a permutation on $\ZZ_p$ and $k = \frac{p-1}2$.  In particular, we have merely pushed
the roots around.

Similarily,
$$
x^{p^k}-x = x(x^\frac{p^k-1}2 - 1)(x^\frac{p^k-1}2 + 1)
$$
is the product of *all* monic polynomials of degree $k$.  Thereby
$$
(x^\frac{p^k-1}2 - 1)
$$
is a product of *half* the monic polynomials of degree $k$.

##### Theorem
Suppose $g_k \in \ZZ_p[x]$ is the product of monic polynomials of degree $k$.   Let $w$ be a *random* monic polynomial of degree $k$.  Then
$$
\mbox{Prob}\left[\gcd(g_k, w^\frac{p^k-1}2 - 1) \not\in \left\{1, g_k \right\}\right] > \frac12 - \frac 1 {2p^2} \geq \frac49.
$$

Notice this means we have an algorithm for splitting the groups returned by the distinct degree factorization.

In [52]:
dds

5-element Vector{Polynomial}:
 16⋅x^1 + 9⋅x^0
 10⋅x^4 + 13⋅x^3 + 15⋅x^2 + 5⋅x^1 + 2⋅x^0
 1⋅x^0
 1⋅x^0
 1⋅x^0

In [53]:
dd_split(dds[2],2,prime)

2-element Vector{Polynomial}:
 1⋅x^2 + 2⋅x^1 + 7⋅x^0
 11⋅x^2 + 11⋅x^1 + 11⋅x^0

### Factoring in $\ZZ_p[x]$

All that remains is to use utilize the previous two algorithms to factor an arbitrary polynomial from $\ZZ_p[x]$.

In [9]:
#Taken from example_script.jl

prime = 17
p = mod((7x^3 + 2x^2 + 8x + 1)*(x^2+x+1),prime)
println("Will factor this polynomial (mod $prime): ", p)
factorization = factor(p,prime)
println("Here is the factorization: ", factorization)

pr = mod(expand_factorization(factorization),prime)
println("Reconstructing: ", pr)


Will factor this polynomial (mod 17): 7⋅x^5 + 9⋅x^4 + 11⋅x^2 + 9⋅x^1 + 1⋅x^0
Here is the factorization: Tuple{Polynomial, Int64}[(1⋅x^1 + 8⋅x^0, 1), (1⋅x^2 + 1⋅x^1 + 1⋅x^0, 1), (1⋅x^2 + 2⋅x^1 + 7⋅x^0, 1), (7⋅x^0, 1)]
Reconstructing: 7⋅x^5 + 9⋅x^4 + 11⋅x^2 + 9⋅x^1 + 1⋅x^0


## Optimizations

For your project you are tasked with two optimizations.

### PowMod

**Question:** How do we compute $\gcd(a,x^{p^k}-x)$ when $k$ large?

*Answer:* Use binary powering with remainder.

Consider that there is a **foolish** way of calculating $8^{17^3} \mod 17 \ldots$
$$
8^{17^3} = 11417981541647679048466287755595961091061972992 \mod 17 = 9 \mod 17
$$
Here we are doing $8^{17^3}$ in $\ZZ$ (not in $\ZZ_p$).  *The first optimization is to fix this* by doing:
$$
(\cdots((8 \cdot 8 \mod 17) \cdot 8 \mod 17) \cdot \cdots \cdot 8 \mod 17)
$$
instead.

Second, we want to *many* less multiplications than $17^3$ many.  We can accomplish this via **repeated squaring** (also called binary powering).  Suppose we want to do $w^{103} \mod m$.  First notice
$$
103 = 64 + 32 + 4 + 2 + 1 = (1100111)
$$
where $(x)_2$ is the binary representaiton of $x$.  Thereby
$$
w^{103} \mod p = (w^{64}) \cdot (w^{32}) \cdot (w^4) \cdot (w^2) \cdot w^1 \mod m
$$
So we just need to calculate
$$
\begin{align*}
&& s &\gets w \\
w^2 \mod a && s &\gets s^2 \mod a \\
w^4 \mod a && s &\gets s^2 \mod a \\
w^{16} \mod a && s &\gets s^2 \mod a \\
w^{32} \mod a && s &\gets s^2 \mod a \\
w^{64} \mod a && s &\gets s^2 \mod a 
\end{align*}
$$
So, instead of doing $103$ multiplications we are doing less than $2 \cdot 5 = 10$ mutiplications (5 to get the squares, then worst case multiply those 5 squares together).

Returning to $8^{17^3}$ we have 
$$17^3 = (1001100110001)_2 = 2^{12} + 2^9 + 2^8 + 2^5 + 2^4 + 2^0$$ 
So here we do 12 multiplications (to get the powers of two by squaring) and then 5 more to get
$$
8^{2^{12}} \cdot 8^{2^9}  \cdot 8^{2^8}  \cdot 8^{2^5}  \cdot 8^{2^4} \cdot 8^{2^0}.
$$

For comparison, the naive method requires $17^3 = 4913$ multiplications.

#### Task
Implement a `PowMod(x, m, p)` which computes $x^m \mod p$ using repeated squaring in $\ZZ_p$.

### Multiplication by Chinese Remainder Theorem

Let $a = a_nx^n + a_{n-1}x^{n-1} + \cdots + a_1 x + a_0$.  Define the **height** of a polynomial by
$$
||a||_{\infty} := \max{|a_0|, \ldots, |a_n|}.
$$
So, for instance $||2x^2-5||_{\infty} = 5$.

Further let 
$$\# a := \mbox{number of nonzero terms of }a.$$  
So, for instance $\# 2x^2-5 = 2 \leq \deg(2x^2-5) + 1$.

Consider that, if we intende to use CRT to cacluate the product $c = a \cdot b$ of $a,b \in \ZZ_p[x]$ we need $M = p_1,\ldots,p_\ell$ satisfying
$$
M = p_1 \cdot \cdots \cdot p_\ell > 2 \cdot ||c||_\infty.
$$

It is *twice* $||c||_\infty$ to account for negative ceofficients.  For instance, $3x^2 - 3x +1$ requires coefficients in $\{ -3,-2,-1,0,1,2,3\}$ which is $\ZZ_7$ and not $\ZZ_5$.

##### Proposition
Let $a,b \in \ZZ[x]$.
$$
||c||_{\infty} \leq ||a||_{\infty} \cdot ||b||_{\infty} \cdot \min(\# a,\, \# b)
$$

*Proof* (Sketch)
Do a worst case analysis where you mutiply $a = \alpha x^n + \alpha x^{n-1} + \cdots + \alpha$ by  $b = \beta x^n + \beta x^{n-1} + \cdots + \beta$ and see what the largest coefficient will end up being.

##### Example 
Let
\begin{align*}
a = 3x - 4, && b = 6x+5, && c = 18^2-9x-20
\end{align*}
and notice $ab = c$.

$$||c||_\infty \leq 4 \cdot 6 \cdot \min(2,2) = 48$$

$$
\begin{align*}
& && 3a_i && b_i && c_i \\[1em]
\hline
p_1 &= 5 && 3x + 1 && 1x + 0 && 3x^2 + 1x + 0 \\[1.5em]
\hline
p_2 &= 7 && 3x + 3 && 6x + 5 && 4x^2 + 5x + 1 \\[1.5em]
\hline
p_3 &= 3 && 0x + 2 && 0x + 2 && 0x^2 + 0x + 1 \\[1.5em]
\hline
\end{align*}
$$

Suppose you have defined a function:

    CRT([u1, u2, ..., um], [p1, p2, ..., pm]) = u  such that u == uk mod pk

Let the **symmetric mod** be given by

    smod(a, m) = (a mod m) if (a mod m) > m//2 else (a mod m) - m

For instance, 

    smod(0, 7) = 0
    smod(1, 7) = 1
    smod(2, 7) = 2
    smod(3, 7) = 3
    smod(4, 7) = -3
    smod(5, 7) = -2
    smod(6, 7) = -1